In [19]:
import numpy as np
import scipy.stats
from scipy.stats import chi2_contingency, ncx2


contingency_table = np.array([[18201, 14293],[2696, 2488]], dtype=np.int64) # This definition is very important for large values 


contingency_table = np.array([
    [225, 53, 206],
    [3, 1, 12]
])

confidence_level = 0.95


# Function from Utils
def calculate_p_value_from_chi_score(chi_score, df):
    p_value = scipy.stats.chi2.sf((abs(chi_score)), df)
    return min(float(p_value), 0.99999)

confidence_level = 0.95

def ncp_ci(chival, df, conf):
    def low_ci(chival, df, conf):
            bounds = [0.001, chival / 2, chival]
            ulim = 1 - (1 - conf) / 2
            while ncx2.cdf(chival, df, bounds[0]) < ulim:
                return [0, ncx2.cdf(chival, df, bounds[0])]
            while (diff := abs(ncx2.cdf(chival, df, bounds[1]) - ulim)) > 0.00001:
                bounds = [bounds[0], (bounds[0] + bounds[1]) / 2, bounds[1]] if ncx2.cdf(chival, df, bounds[1]) < ulim else [bounds[1], (bounds[1] + bounds[2]) / 2, bounds[2]]
            return [bounds[1]]
    def high_ci(chival, df, conf):
        # This first part finds upper and lower starting values.
        uc = [chival, 2 * chival, 3 * chival]
        llim = (1 - conf) / 2
        while ncx2.cdf(chival, df, uc[0]) < llim: uc = [uc[0] / 4, uc[0], uc[2]]
        while ncx2.cdf(chival, df, uc[2]) > llim: uc = [uc[0], uc[2], uc[2] + chival]

        diff = 1
        while diff > 0.00001:
            uc = [uc[0], (uc[0] + uc[1]) / 2, uc[1]] if ncx2.cdf(chival, df, uc[1]) < llim else [uc[1], (uc[1] + uc[2]) / 2, uc[2]]
            diff = abs(ncx2.cdf(chival, df, uc[1]) - llim)
            lcdf = ncx2.cdf(chival, df, uc[1])   
        return uc[1]
    low_ncp = low_ci(chival, df, conf)
    high_ncp = high_ci(chival, df, conf)

    return low_ncp[0], high_ncp

def likelihood_ratio(contingency_table):
        Observed = contingency_table
        Sum_Product = np.sum(Observed)
        Expected = np.outer(np.sum(Observed, axis=1), np.sum(Observed, axis=0)) / Sum_Product
        likelihood_ratio = 0
        for i in range(2):
            for j in range(2):
                likelihood_ratio += Observed[i, j] * np.log(Observed[i, j] / Expected[i, j])
        likelihood_ratio = 2 * likelihood_ratio
        p_value = 1 - scipy.stats.chi2.cdf(likelihood_ratio, 1)
        return [likelihood_ratio, p_value]

# Another definition of BM effect size that returns the maximum chi square matrix that will allow the calucaltion of chi square from the matrix while ignoring divisions by zer0
def Berry_Mielke_Maximum_Corrected_Cramer_V_output_matrix(matrix):
    Observed_Chi_Square = chi2_contingency(matrix).statistic
    r, c = matrix.shape 
    sum_of_rows_vector = matrix.sum(axis=1)  
    sum_of_cols_vector = matrix.sum(axis=0)  
    NR = sum_of_rows_vector.sum()  
    NC = sum_of_cols_vector.sum() 

    matrix = np.zeros((r, c)) 
    x = np.where(np.isin(sum_of_cols_vector, sum_of_rows_vector), np.argmax(np.isin(sum_of_rows_vector, sum_of_cols_vector), axis=0) + 1, np.nan)
    y = np.where(np.isin(sum_of_rows_vector, sum_of_cols_vector), np.argmax(np.isin(sum_of_cols_vector, sum_of_rows_vector), axis=0) + 1, np.nan)
    x = x[~np.isnan(x)].astype(int) - 1  
    y = y[~np.isnan(y)].astype(int) - 1  

    matrix[x,y] = sum_of_rows_vector[x]
    sum_of_rows_vector[x] = sum_of_rows_vector[x]-sum_of_rows_vector[x] 
    sum_of_cols_vector[y] = sum_of_cols_vector[y]-sum_of_cols_vector[y] 

    while NR > 0 and NC > 0:
        NR = np.sum(sum_of_rows_vector)  
        NC = np.sum(sum_of_cols_vector)  
        x = np.argmax(sum_of_rows_vector)  
        y = np.argmax(sum_of_cols_vector)  
        z = min(sum_of_rows_vector[x], sum_of_cols_vector[y]) 
        matrix[x, y] = z
        sum_of_rows_vector[x] -= z
        sum_of_cols_vector[y] -= z

    Maximum_Corrected_Cramers_v = Observed_Chi_Square

    return matrix


def multilevel_contingency_tables(contingency_table, confidence_level):

    # Preperations:   
    Sample_Size = np.sum(contingency_table)
    Number_of_rows = np.size(contingency_table, axis = 0)
    Number_of_Coloumns = np.size(contingency_table, axis = 1)
    matrix = np.array(contingency_table)

    # Calcualte Chi Square and its significance
    row_totals = np.sum(contingency_table, axis=1)
    col_totals = np.sum(contingency_table, axis=0)
    expected = np.multiply.outer(row_totals, col_totals) / Sample_Size
    chi_squared = np.sum( ((contingency_table - expected)**2) / expected)

    degrees_of_freedom_chi_square = (Number_of_Coloumns-1) * (Number_of_rows-1)
    p_value = calculate_p_value_from_chi_score(chi_squared, degrees_of_freedom_chi_square)

    # Effect Sizes for R X C tables
    phi_square = chi_squared/Sample_Size 
    phi = np.sqrt(phi_square) # This is also knows as Cohens w
    Cramer_V = (phi_square / (min(Number_of_Coloumns-1, Number_of_rows-1)))**0.5
    Tschuprows_T = (phi_square / ( (Number_of_Coloumns-1) * (Number_of_rows-1))**0.5)**0.5
    Pearsons_Contingency_Coefficient = np.sqrt(chi_squared/(chi_squared+Sample_Size)) 
    likelihood_ratio_statistic, likelihood_ratio_pvalue = likelihood_ratio(matrix)

    # Bias Corrected Effect Sizes
    Phi_Square_Bias_Corrected = phi_square - (1 / (Sample_Size-1)) * (Number_of_Coloumns - 1) * (Number_of_rows - 1)
    chi_square_Bias_corrected = Phi_Square_Bias_Corrected * Sample_Size
    Phi_Bias_Corrected = np.sqrt(Phi_Square_Bias_Corrected)
    Number_of_rows_Corrected = Number_of_rows - (1/(Sample_Size-1)) * (Number_of_rows-1)**2
    Number_of_coloumns_Corrected = Number_of_Coloumns - (1/(Sample_Size-1)) * (Number_of_Coloumns-1)**2
    
    Bias_Corrected_Tschuprows_T = (Phi_Square_Bias_Corrected / ( (Number_of_coloumns_Corrected-1) * (Number_of_rows_Corrected-1))**0.5)**0.5 # Bergsma, 2013
    Bias_Corrected_Cramer_V = (Phi_Square_Bias_Corrected / (min(Number_of_coloumns_Corrected-1, Number_of_rows_Corrected-1)))**0.5 # Bergsma, 2013
    Bias_Corrected_Contingency_Coefficeint = np.sqrt(Phi_Square_Bias_Corrected/ (Phi_Square_Bias_Corrected+1))

    # Maximum Corrected Effect Sizes
    k = min(Number_of_coloumns_Corrected, Number_of_rows_Corrected)
    Maximum_Contingency_Coefficient =  np.sqrt((k-1) / k) #Sakoda, 1977
    Maximum_Corrected_Contingency_Coefficient = Pearsons_Contingency_Coefficient / Maximum_Contingency_Coefficient
    Maximum_Tschuprows_T = np.sqrt(np.sqrt(((k-1) / (max(Number_of_Coloumns-1, Number_of_rows-1)))**0.25))
    Maximum_Corrected_Tschuprows_T = Tschuprows_T / Maximum_Tschuprows_T
    
    #Maximum_Corrected_Chi_Square = Berry_Mielke_Maximum_Corrected_Cramer_V(contingency_table) # Berry and mielke Lambda
    Maximum_Corrected_Matrix = Berry_Mielke_Maximum_Corrected_Cramer_V_output_matrix(matrix) # Berry and mielke Lambda
    row_totals_max = np.sum(Maximum_Corrected_Matrix, axis=1)
    col_totals_max = np.sum(Maximum_Corrected_Matrix, axis=0)
    total_max = np.sum(Maximum_Corrected_Matrix)
    expected_max = np.outer(row_totals_max, col_totals_max) / total_max
    
    # This Method removes zeros for the calucaltion to avoid division by zero when one clacualte expected values 
    observeved = Maximum_Corrected_Matrix.flatten()
    expected_max = np.array(expected_max).flatten()
    zero_positions = np.where((observeved == 0) & (expected_max == 0))[0]
    observeved = np.delete(observeved, zero_positions)
    expected_max = np.delete(expected_max, zero_positions)
    chi_squared_max = np.sum((observeved - expected_max)**2 / expected_max)
    Maximum_Corrected_cramers_v = chi_squared / chi_squared_max
    

    # Calculation of the Variance
    Probabilities_Table = contingency_table/Sample_Size
    Squared_Probabilities_Table = Probabilities_Table**2
    Cubic_Probability_Table = Probabilities_Table**3

    column_marginals = np.sum(Probabilities_Table, axis=0)
    row_marginals = np.sum(Probabilities_Table, axis=1)
    marginal_products = np.outer(row_marginals, column_marginals)
    Corrected_Probabilities_Table = Squared_Probabilities_Table / marginal_products

    column_marginals_Squared = np.sum(Probabilities_Table, axis=0)**2
    row_marginals_Squared = np.sum(Probabilities_Table, axis=1)**2
    marginal_products_Corrected= np.outer(row_marginals_Squared, column_marginals_Squared)
    Corrected_Squared_Probabilities_Table = Cubic_Probability_Table / marginal_products_Corrected

    row_marginals_corrected = np.sum(Corrected_Probabilities_Table, axis=1)
    coloumn_marginals_corrected = np.sum(Corrected_Probabilities_Table, axis=0)
    marginal_products_corrected_table = np.outer(row_marginals_corrected,coloumn_marginals_corrected )

    term1 = np.sum(Corrected_Squared_Probabilities_Table)
    term2 = np.sum(row_marginals_corrected**2 / row_marginals)
    term3 = np.sum(coloumn_marginals_corrected**2 / column_marginals)  
    term4 = np.sum(marginal_products_corrected_table)


    # Confidence Intervals using the asymptotic standard error  (Bishop et al., 1975)
    z_crit = scipy.stats.norm.ppf(confidence_level + ((1 - confidence_level) / 2))
    Standart_deviation_phi_square = np.sqrt((4*term1 - 3*term2 - 3*term3 + 2*term4) / Sample_Size)
    ci_lower_phi = phi - Standart_deviation_phi_square*z_crit
    ci_upper_phi = phi + Standart_deviation_phi_square*z_crit


    # Contingency Coefficient
    Contingency_Coefficient_Standard_deviation = (1 / (2*phi*(1+phi_square)**(3/2)))*Standart_deviation_phi_square
    Bias_Corrected_Contingency_Coefficient_Standard_deviation = 0 if Bias_Corrected_Cramer_V == 0 else (1 / (2*np.sqrt(Phi_Square_Bias_Corrected)*(1+Phi_Square_Bias_Corrected)**(3/2)))*Standart_deviation_phi_square

    q = min(Number_of_Coloumns,Number_of_rows)
    Maximum_Corrected_CC_Standard_Error = np.sqrt(  (q / (q-1)) * Contingency_Coefficient_Standard_deviation**2)
    ci_lower_cc = Pearsons_Contingency_Coefficient - Contingency_Coefficient_Standard_deviation*z_crit
    ci_upper_cc = Pearsons_Contingency_Coefficient + Contingency_Coefficient_Standard_deviation*z_crit
    ci_lower_cc_corrected = Maximum_Corrected_Contingency_Coefficient - Maximum_Corrected_CC_Standard_Error*z_crit
    ci_upper_cc_corrected = Maximum_Corrected_Contingency_Coefficient + Maximum_Corrected_CC_Standard_Error*z_crit

    # Cramer's V 
    Cramer_V_standart_deviation = (1 / (2 * (min(Number_of_Coloumns-1, Number_of_rows-1))**0.5 * Cramer_V)) * Standart_deviation_phi_square
    Bias_Corrected_Cramer_V_standard_deviation = (1 / (2 * (min(Number_of_coloumns_Corrected-1, Number_of_rows_Corrected-1))**0.5 * Bias_Corrected_Cramer_V)) * Standart_deviation_phi_square
    ci_lower_cramer = Cramer_V - Cramer_V_standart_deviation*z_crit
    ci_upper_cramer = Cramer_V + Cramer_V_standart_deviation*z_crit
    ci_lower_cramer_corrected = Bias_Corrected_Cramer_V - Bias_Corrected_Cramer_V_standard_deviation*z_crit
    ci_upper_crame_corrected = Bias_Corrected_Cramer_V + Bias_Corrected_Cramer_V_standard_deviation*z_crit

    # Tschuprows T
    Tschuprows_T_Standard_deviation = (1 / (2 * (Number_of_Coloumns-1) * (Number_of_rows-1) * Tschuprows_T) ) * Standart_deviation_phi_square
    Bias_Corrected_Tschuprows_T_Standard_deviation = (1 / (2 * (Number_of_coloumns_Corrected-1) * (Number_of_rows_Corrected-1) * Bias_Corrected_Tschuprows_T) ) * Standart_deviation_phi_square
    ci_lower_tschuprows = Tschuprows_T - Tschuprows_T_Standard_deviation*z_crit
    ci_upper_tschuprows = Tschuprows_T + Tschuprows_T_Standard_deviation*z_crit
    ci_lower_tschuprows_corrected = Bias_Corrected_Tschuprows_T - Bias_Corrected_Tschuprows_T_Standard_deviation*z_crit
    ci_upper_tschuprows_corrected = Bias_Corrected_Tschuprows_T + Bias_Corrected_Tschuprows_T_Standard_deviation*z_crit

    
    
    # Confidence Interals using the non-central distribution
    lower_ci_ncp, upper_ci_ncp = ncp_ci(chi_squared ,degrees_of_freedom_chi_square, confidence_level)
    lower_ci_ncp_bias_corrected, upper_ci_ncp_bias_corrected= ncp_ci(chi_square_Bias_corrected, degrees_of_freedom_chi_square, confidence_level)
    lower_ci_ncp_bias_corrected, upper_ci_ncp_bias_corrected = max(lower_ci_ncp_bias_corrected,0), max(upper_ci_ncp_bias_corrected,0)

    lower_ncp_max_chi_square, upper_ncp_max_chi_square = ncp_ci(chi_squared_max, degrees_of_freedom_chi_square, confidence_level)

    lower_ncp_phi_square = lower_ci_ncp_bias_corrected / Sample_Size
    upper_ncp_phi_square = upper_ci_ncp_bias_corrected / Sample_Size

    
    # NCP Confidence interval for Contingency Coefficient
    ci_lower_cc_ncp = np.sqrt(lower_ci_ncp / (lower_ci_ncp + Sample_Size))
    ci_upper_cc_ncp = np.sqrt(upper_ci_ncp / (upper_ci_ncp + Sample_Size))
    ci_lower_cc_bias_corrected_ncp = np.sqrt(lower_ci_ncp_bias_corrected / (lower_ci_ncp + Sample_Size))
    ci_upper_cc_bias_corrected_ncp = np.sqrt(upper_ci_ncp_bias_corrected / (upper_ci_ncp + Sample_Size))
    ci_lower_cc_max_corrected_ncp = np.sqrt(((lower_ncp_phi_square * q) / ((q-1)* (1+lower_ncp_phi_square))))
    ci_upper_cc_max_corrected_ncp = np.sqrt(((upper_ncp_phi_square * q) / ((q-1)* (1+upper_ncp_phi_square))))
       
    # NCP Confidence interval for Cramer V
    ci_ncp_lower_cramer = np.sqrt(lower_ci_ncp / (Sample_Size * q))
    ci_ncp_upper_cramer = np.sqrt(upper_ci_ncp / (Sample_Size * q))
    ci_ncp_lower_cramer_bias_corrected = np.sqrt(lower_ci_ncp_bias_corrected / (Sample_Size * q))
    ci_ncp_upper_cramer_bias_corrected = np.sqrt(upper_ci_ncp_bias_corrected / (Sample_Size * q))
    ci_ncp_lower_cramer_max_corrected = lower_ci_ncp / lower_ncp_max_chi_square
    ci_ncp_upper_cramer_max_corrected = upper_ci_ncp / upper_ncp_max_chi_square


    # NCP confidence Intervals for Tschuprow's T
    ci_ncp_lower_Tschuprow = np.sqrt(lower_ci_ncp / (Sample_Size * np.sqrt(degrees_of_freedom_chi_square)))
    ci_ncp_upper_Tschuprow = np.sqrt(upper_ci_ncp / (Sample_Size * np.sqrt(degrees_of_freedom_chi_square)))
    ci_ncp_lower_Tschuprow_bias_corrected = np.sqrt(lower_ci_ncp_bias_corrected / (Sample_Size * np.sqrt(degrees_of_freedom_chi_square)))
    ci_ncp_upper_Tschuprow_bias_corrected = np.sqrt(upper_ci_ncp_bias_corrected / (Sample_Size * np.sqrt(degrees_of_freedom_chi_square)))


    # 3. Uncertainty Coefficient - Needs to be in a Contingency Table
    Sum_Of_Rows = np.sum(contingency_table, axis=1)
    Sum_Of_Columns = np.sum(contingency_table, axis=0)
    HX = -np.sum((Sum_Of_Rows * np.log(Sum_Of_Rows / Sample_Size)) / Sample_Size)
    HY = -np.sum((Sum_Of_Columns * np.log(Sum_Of_Columns / Sample_Size)) / Sample_Size)
    HXY = -np.sum(contingency_table * np.log(contingency_table / Sample_Size)) / Sample_Size
    
    # Calculate Effect Size UC 
    Uncertainty_Coefficient_Symmetric = 2 * (HX + HY - HXY) / (HX + HY)
    Uncertainty_Coefficient_Rows = (HX + HY - HXY) / HX
    Uncertainty_Coefficient_Columns = (HX + HY - HXY) / HY

    # Calculate the Asymptotic Standard Errors (Standard Errors)
    Standard_Error_Symmetric = np.sqrt((4 * np.sum(contingency_table * (HXY * np.log(np.outer(Sum_Of_Rows, Sum_Of_Columns) / Sample_Size ** 2) - (HX + HY) * np.log(contingency_table / Sample_Size)) ** 2) / (Sample_Size ** 2 * (HX + HY) ** 4)))
    Standard_Error_Rows = np.sqrt(np.sum(contingency_table * (HX * np.log(contingency_table / Sum_Of_Columns) + (HY - HXY) * np.log(Sum_Of_Rows / Sample_Size)[:, np.newaxis]) ** 2) / (Sample_Size ** 2 * HX ** 4))
    Standard_Error_Columns = np.sqrt(np.sum(contingency_table * (HY * np.log(contingency_table / Sum_Of_Rows[:, np.newaxis]) + (HX - HXY) * np.log(Sum_Of_Columns / Sample_Size)) ** 2) / (Sample_Size ** 2 * HY ** 4))

    # Calculate p_values        
    Z_value_Symmetric = Uncertainty_Coefficient_Symmetric / Standard_Error_Symmetric
    Z_value_Rows = Uncertainty_Coefficient_Rows / Standard_Error_Rows
    Z_value_Columns = Uncertainty_Coefficient_Columns / Standard_Error_Columns

    # Confidence Intervals
    Zcrit = 1 - (1 - confidence_level) / 2
    Symmetric_CIs = Zcrit * np.sqrt(Standard_Error_Symmetric) * np.array([-1, 1]) + Uncertainty_Coefficient_Symmetric
    Rows_CIs = Zcrit * np.sqrt(Standard_Error_Rows) * np.array([-1, 1]) + Uncertainty_Coefficient_Rows
    Columns_CIs = Zcrit * np.sqrt(Standard_Error_Columns) * np.array([-1, 1]) + Uncertainty_Coefficient_Columns 


    results = {}

    # Add the relevant parameters to the dictionary
    results["Sample Size"] = Sample_Size
    results["Number of Rows"] = Number_of_rows
    results["Number of Columns"] = Number_of_Coloumns
    results["____________________________________________"] = ''  

    results["Chi Square"] = round(chi_squared, 4)
    results["Degrees of Freedom Chi Square"] = round(degrees_of_freedom_chi_square, 4)
    results["p_value chi square"] = round(p_value, 10) # this one is suitable for all kind of effect sizes

    results["___________________________________________"] = ''  
    results["Phi Square"] = round(phi_square, 10)
    results["Phi / Cohen's w"] = round(phi, 4)
    results["Cramer V"] = round(Cramer_V, 4)
    results["Tschuprow's T"] = round(Tschuprows_T, 7)
    results["Pearson's Contingency Coefficient"] = round(Pearsons_Contingency_Coefficient, 4)
    results["Likelihood Ratio"] = round(likelihood_ratio_statistic, 4)
    results["Likelihood Ratio p_value"] = round(likelihood_ratio_pvalue, 4)

    # Bias Corrected Measures
    results["__________________________________________________"] = ''  
    results["Adjusted Phi"] = round(Phi_Square_Bias_Corrected, 7)
    results["Adjusted Contingency Coefficient"] = round(Bias_Corrected_Contingency_Coefficeint, 4)
    results["Adjusted Cramer's V"] = round(Bias_Corrected_Cramer_V, 7)
    results["Adjusted Tschuprow's T"] = round(Bias_Corrected_Tschuprows_T, 7)

    #Maximum Corrected Measures
    results["__________________________________________"] = ''  
    results["Maximum Corrected Contingency Coefficient (Sakoda, 1977)"] = round(Maximum_Corrected_Contingency_Coefficient, 4)
    results["Maximum Corrected Tschuprow's T"] = round(Maximum_Corrected_Tschuprows_T, 7)
    results["Maximum Corrected Cramers V (Berry, Mielke, Jhonston)"] = round(Maximum_Corrected_cramers_v, 7)

    # Standrd dviations
    results["_________________________________________"] = ''  
    results["Standard Deviation of Phi Square"] = round(Standart_deviation_phi_square, 4)
    results["Standard Deviation of Contingency Coefficient"] = round(Contingency_Coefficient_Standard_deviation, 4)
    results["Standard Deviation of Maximum Corrected Contingency Coefficient"] = round(Maximum_Corrected_CC_Standard_Error, 4)
    results["Standard Deviation of Cramer V"] = round(Cramer_V_standart_deviation, 4)
    results["Standard Deviation of Tschuprows T"] = round(Tschuprows_T_Standard_deviation, 4)

    # Asymptotic CI's
    results["______________________________________________"] = ''
    results["CI Phi / Cohens w"] = f"({round(ci_lower_phi, 4)}, {round(ci_upper_phi, 4)})"
    results["CI maximum corrected Contingency Coefficient"] = f"({round(ci_lower_cc_corrected, 4)}, {round(ci_upper_cc_corrected, 4)})"
    results["CI Contingency Coefficient"] = f"({round(ci_lower_cc, 4)}, {round(ci_upper_cc, 4)})"
    results["CI maximum corrected Contingency Coefficient"] = f"({round(ci_lower_cc_corrected, 4)}, {round(ci_upper_cc_corrected, 4)})"
    results["CI Cramer V"] = f"({round(ci_lower_cramer, 4)}, {round(ci_upper_cramer, 4)})"
    results["CI bias corrected Cramer V"] = f"({round(ci_lower_cramer_corrected, 4)}, {round(ci_upper_crame_corrected, 4)})"
    results["CI Tschuprow's T"] = f"({round(ci_lower_tschuprows, 4)}, {round(ci_upper_tschuprows, 4)})"
    results["CI bias corrected Tschuprows T"] = f"({round(ci_lower_tschuprows_corrected, 4)}, {round(ci_upper_tschuprows_corrected, 4)})"

    # Non Central CI's
    results["______________________________________________________"] = '' 
    results["NCP CI Contingency Coefficient"] = f"({round(ci_lower_cc_ncp, 4)}, {round(ci_upper_cc_ncp, 4)})"
    results["NCP CI Cramer V"] = f"({round(ci_ncp_lower_cramer, 4)}, {round(ci_ncp_upper_cramer, 4)})"
    results["NCP CI Tschuprow's T"] = f"({round(ci_ncp_lower_Tschuprow, 4)}, {round(ci_ncp_upper_Tschuprow, 4)})"
    results["NCP CI Contingency Coefficient Bias corrected"] = f"({round(ci_lower_cc_bias_corrected_ncp, 4)}, {round(ci_upper_cc_bias_corrected_ncp, 4)})"
    results["NCP CI Cramer's V Bias corrected"] = f"({round(ci_ncp_lower_cramer_bias_corrected, 4)}, {round(ci_ncp_upper_cramer_bias_corrected, 4)})"
    results["NCP CI Tschuprow's T Bias corrected"] = f"({round(ci_ncp_lower_Tschuprow_bias_corrected, 4)}, {round(ci_ncp_upper_Tschuprow_bias_corrected, 4)})"
    results["NCP CI Contingency Coefficient Max corrected"] = f"({round(ci_lower_cc_max_corrected_ncp, 4)}, {round(ci_upper_cc_max_corrected_ncp, 4)})"
    results["NCP CI Cramer's V Max corrected"] = f"({round(ci_ncp_lower_cramer_max_corrected, 4)}, {round(ci_ncp_upper_cramer_max_corrected, 4)})"

    # Uncertinty Coefficient
    results["Uncertainty Coefficient (Symmetric)"] = Uncertainty_Coefficient_Symmetric
    results["Uncertainty Coefficient (Rows)"] = Uncertainty_Coefficient_Rows
    results["Uncertainty Coefficient (Columns)"] = Uncertainty_Coefficient_Columns
    results["Uncertainty Coefficient Standard Error (Symmetric)"] = Standard_Error_Symmetric
    results["Uncertainty Coefficient Standard Error (Rows)"] = Standard_Error_Rows
    results["Uncertainty Coefficient Standard Error (Columns)"] = Standard_Error_Columns
    results["Uncertainty Coefficient Z-value (Symmetric)"] = Z_value_Symmetric
    results["Uncertainty Coefficient Z-value (Rows)"] = Z_value_Rows
    results["Uncertainty Coefficient Z-value (Columns)"] = Z_value_Columns
    results["Uncertainty Coefficient Confidence Intervals (Symmetric)"] = Symmetric_CIs
    results["Uncertainty Coefficient Confidence Intervals (Rows)"] = Rows_CIs
    results["Uncertainty Coefficient Confidence Intervals (Columns)"] = Columns_CIs

   
    result_str = "\n".join([f"{key}: {value}" for key, value in results.items()])
    return result_str 




Sample Size: 500
Number of Rows: 2
Number of Columns: 3
____________________________________________: 
Chi Square: 6.6678
Degrees of Freedom Chi Square: 2
p_value chi square: 0.0356532724
___________________________________________: 
Phi Square: 0.0133356574
Phi / Cohen's w: 0.1155
Cramer V: 0.1155
Tschuprow's T: 0.0971068
Pearson's Contingency Coefficient: 0.1147
Likelihood Ratio: 3.715
Likelihood Ratio p_value: 0.0539
__________________________________________________: 
Adjusted Phi: 0.0093276
Adjusted Contingency Coefficient: 0.0961
Adjusted Cramer's V: 0.0966766
Adjusted Tschuprow's T: 0.0813359
__________________________________________: 
Maximum Corrected Contingency Coefficient (Sakoda, 1977): 0.1623
Maximum Corrected Tschuprow's T: 0.1014188
Maximum Corrected Cramers V (Berry, Mielke, Jhonston): 0.0971567
_________________________________________: 
Standard Deviation of Phi Square: 0.0095
Standard Deviation of Contingency Coefficient: 0.0404
Standard Deviation of Maximum Correc